In [33]:
import pandas as pd
import os
from library import start
from library import clean_for_merge

In [34]:
tea = pd.read_csv(os.path.join(start.data_path, 'tea', 'desc_long.csv'),
                  sep=",")
tea = tea[['district', 'distname', 'year', 
          'cntyname', 'distischarter', 'rating',
          'students_num', 'students_frpl_num',
          'students_black_num', 'students_hisp_num', 'students_white_num',
          'teachers_num', 'teachers_new_num', 'teacher_turnover_num', 
          'teachers_turnover_denom', 'teachers_turnover_ratio',
          'teachers_exp_ave', 'teachers_tenure_ave',
          'teachers_nodegree_num', 'teachers_badegree_num',
          'teachers_msdegree_num', 'teachers_phddegree_num',
          'r_3rd_avescore','r_3rd_numtakers',
          'm_3rd_avescore', 'm_3rd_numtakers',
          'r_4th_avescore', 'r_4th_numtakers',
          'm_4th_avescore', 'm_4th_numtakers',
          'r_5th_avescore', 'r_5th_numtakers',
          'm_5th_avescore', 'm_5th_numtakers',
          'r_6th_avescore', 'r_6th_numtakers',
          'm_6th_avescore', 'm_6th_numtakers',
          'r_7th_avescore', 'r_7th_numtakers',
          'm_7th_avescore', 'm_7th_numtakers',
          'r_8th_avescore', 'r_8th_numtakers', 
          'm_8th_avescore', 'm_8th_numtakers',
          'alg_avescore', 'alg_numtakers', 
          'bio_avescore', 'bio_numtakers',
          'eng1_avescore', 'eng1_numtakers', 
          'eng2_avescore', 'eng2_numtakers',
          'us_avescore', 'us_numtakers']]
tea.head()

,district,distname,year,cntyname,distischarter,rating,students_num,students_frpl_num,students_black_num,students_hisp_num,...,alg_avescore,alg_numtakers,bio_avescore,bio_numtakers,eng1_avescore,eng1_numtakers,eng2_avescore,eng2_numtakers,us_avescore,us_numtakers
0,1902,CAYUGA ISD,yr1112,ANDERSON,N,NaN,590,220,35,30,...,4079.0,61.0,4049.0,60.0,2049.0,62.0,NaN,NaN,3535.0,44.0
1,1903,ELKHART ISD,yr1112,ANDERSON,N,NaN,1271,706,62,100,...,4012.0,68.0,3608.0,80.0,2040.0,79.0,NaN,NaN,NaN,NaN
2,1904,FRANKSTON ISD,yr1112,ANDERSON,N,NaN,733,425,82,76,...,3733.0,34.0,3512.0,32.0,1898.0,37.0,NaN,NaN,3503.0,37.0
3,1906,NECHES ISD,yr1112,ANDERSON,N,NaN,403,163,29,40,...,3885.0,33.0,4056.0,33.0,1968.0,34.0,NaN,NaN,NaN,NaN
4,1907,PALESTINE ISD,yr1112,ANDERSON,N,NaN,3220,2470,903,1255,...,3657.0,160.0,3847.0,196.0,1858.0,196.0,NaN,NaN,3713.0,198.0


In [35]:
laws = pd.read_csv(os.path.join(start.data_path, 'plans', 'doi_final.csv'),
                   sep=",")
laws = laws.drop(['Unnamed: 0', 'level', 'type', 'link', 'p_doi', 'date_p'],
                 axis = 1)
laws = laws.rename({'district': 'distname'}, axis = 1)
laws.head()

,distname,date,reg1_251,reg1_253,reg102_1301,reg102_1303,reg102_1305,reg102_1307,reg102_1309,reg102_1311,...,reg551_083,reg551_084,reg552_023,reg552_024,reg7_056,reg75_600,reg8_210,reg921_406,reg97_1001,reg97_1055
0,Abbott ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,2017.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Clean variables for merge

problems with district name from scraping

In [36]:
tea = tea.pipe(clean_for_merge.resolve_unicode_problems, 'distname')
laws = laws.pipe(clean_for_merge.resolve_unicode_problems, 'distname')

scraped names in title case, but tea all caps. change scraped distname to caps

In [37]:
laws = laws.pipe(clean_for_merge.uppercase_column, 'distname')

sometimes districts named CISD othertimes ISD. Make all ISD

In [38]:
tea = clean_for_merge.replace_column_values(tea, 'distname', 'CISD', 'ISD')
laws = clean_for_merge.replace_column_values(laws, 'distname', 'CISD', 'ISD')

fix district names that don't match

In [39]:
tea = clean_for_merge.sync_district_names(tea, 'distname')
laws = clean_for_merge.sync_district_names(laws, 'distname')

In [40]:
mismatch = clean_for_merge.get_not_in(laws, 'distname', tea, 'distname')
mismatch_list = clean_for_merge.strip_distnum_parens(list(mismatch.distname))

In [41]:
df = clean_for_merge.distnum_in_paren(
    tea[[elem in mismatch_list for elem in tea.distname]])
df

/Users/kylieleblancKylie/domino/dofis/code/clean/library/clean_for_merge.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ')'


,district,distname,year,cntyname,distischarter,rating,students_num,students_frpl_num,students_black_num,students_hisp_num,...,alg_avescore,alg_numtakers,bio_avescore,bio_numtakers,eng1_avescore,eng1_numtakers,eng2_avescore,eng2_numtakers,us_avescore,us_numtakers
80,15905,EDGEWOOD ISD (015905),yr1112,BEXAR,N,NaN,11822,11425,110,11634,...,3595.0,617.0,3665.0,636.0,1853.0,658.0,1658.0,7.0,3622.0,207.0
90,15915,NORTHSIDE ISD (015915),yr1112,BEXAR,N,NaN,97204,52226,5974,66384,...,3984.0,6896.0,4005.0,6639.0,2031.0,6849.0,1793.0,268.0,3664.0,786.0
115,19913,HUBBARD ISD (019913),yr1112,BOWIE,N,NaN,97,61,0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,39905,MIDWAY ISD (039905),yr1112,CLAY,N,NaN,94,44,0,5,...,4012.0,5.0,3757.0,5.0,1912.0,5.0,NaN,NaN,NaN,NaN
212,43914,WYLIE ISD (043914),yr1112,COLLIN,N,NaN,12879,3546,1743,2729,...,4017.0,783.0,4081.0,877.0,2043.0,894.0,NaN,NaN,NaN,NaN
294,57911,HIGHLAND PARK ISD (057911),yr1112,DALLAS,N,NaN,6770,0,30,320,...,4284.0,512.0,4377.0,508.0,2151.0,511.0,2059.0,272.0,4345.0,270.0
607,109905,HUBBARD ISD (109905),yr1112,HILL,N,NaN,403,278,85,43,...,3717.0,26.0,3817.0,23.0,2025.0,22.0,NaN,NaN,3509.0,26.0
754,145902,CENTERVILLE ISD (145902),yr1112,LEON,N,NaN,689,261,57,80,...,3732.0,57.0,3800.0,62.0,1973.0,65.0,NaN,NaN,3397.0,22.0
809,161903,MIDWAY ISD (161903),yr1112,MCLENNAN,N,NaN,7180,2147,811,1400,...,3990.0,505.0,4067.0,496.0,2058.0,491.0,1761.0,97.0,NaN,NaN
954,188903,HIGHLAND PARK ISD (188903),yr1112,POTTER,N,NaN,922,553,35,281,...,3818.0,59.0,3783.0,63.0,1954.0,64.0,NaN,NaN,3772.0,53.0


In [42]:
tea.loc[(tea['distname'].isin(mismatch_list)), 'distname'] = (
    tea.loc[(tea['distname'].isin(mismatch_list))]
        .pipe(clean_for_merge.distnum_in_paren)['distname']
)

/Users/kylieleblancKylie/domino/dofis/code/clean/library/clean_for_merge.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ')'


create number and non-number version of every district in tea. Then we can merge with correct version

In [43]:
tea.head()

,district,distname,year,cntyname,distischarter,rating,students_num,students_frpl_num,students_black_num,students_hisp_num,...,alg_avescore,alg_numtakers,bio_avescore,bio_numtakers,eng1_avescore,eng1_numtakers,eng2_avescore,eng2_numtakers,us_avescore,us_numtakers
0,1902,CAYUGA ISD,yr1112,ANDERSON,N,NaN,590,220,35,30,...,4079.0,61.0,4049.0,60.0,2049.0,62.0,NaN,NaN,3535.0,44.0
1,1903,ELKHART ISD,yr1112,ANDERSON,N,NaN,1271,706,62,100,...,4012.0,68.0,3608.0,80.0,2040.0,79.0,NaN,NaN,NaN,NaN
2,1904,FRANKSTON ISD,yr1112,ANDERSON,N,NaN,733,425,82,76,...,3733.0,34.0,3512.0,32.0,1898.0,37.0,NaN,NaN,3503.0,37.0
3,1906,NECHES ISD,yr1112,ANDERSON,N,NaN,403,163,29,40,...,3885.0,33.0,4056.0,33.0,1968.0,34.0,NaN,NaN,NaN,NaN
4,1907,PALESTINE ISD,yr1112,ANDERSON,N,NaN,3220,2470,903,1255,...,3657.0,160.0,3847.0,196.0,1858.0,196.0,NaN,NaN,3713.0,198.0


In [44]:
laws.head()

,distname,date,reg1_251,reg1_253,reg102_1301,reg102_1303,reg102_1305,reg102_1307,reg102_1309,reg102_1311,...,reg551_083,reg551_084,reg552_023,reg552_024,reg7_056,reg75_600,reg8_210,reg921_406,reg97_1001,reg97_1055
0,ABBOTT ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABERNATHY ISD,2017.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABILENE ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACADEMY ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ADRIAN ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Merge

In [45]:
len(data)

8514

In [46]:
data = tea.merge(laws, left_on='distname', right_on='distname', how='left', indicator = True)
data.loc[(data['_merge'] == 'both'), 'doi'] = True 
data.loc[(data['_merge'] == 'left_only'), 'doi'] = False 
data.head()

,district,distname,year,cntyname,distischarter,rating,students_num,students_frpl_num,students_black_num,students_hisp_num,...,reg552_023,reg552_024,reg7_056,reg75_600,reg8_210,reg921_406,reg97_1001,reg97_1055,_merge,doi
0,1902,CAYUGA ISD,yr1112,ANDERSON,N,NaN,590,220,35,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1,1903,ELKHART ISD,yr1112,ANDERSON,N,NaN,1271,706,62,100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
2,1904,FRANKSTON ISD,yr1112,ANDERSON,N,NaN,733,425,82,76,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
3,1906,NECHES ISD,yr1112,ANDERSON,N,NaN,403,163,29,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
4,1907,PALESTINE ISD,yr1112,ANDERSON,N,NaN,3220,2470,903,1255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True


In [49]:
laws.distname.nunique(), tea.distname.nunique(), data.distname.nunique()

(781, 1276, 1276)

# Save

In [50]:
data.to_csv(os.path.join(start.data_path, 'clean', 'master_data.csv'),
                  sep=",")